In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Define paths for dataset and output
DATASET_PATH = "./Dataset"  # Replace with your dataset path
OUTPUT_PATH = "./output"  # Replace with desired output path for saving models and plots

def load_path(path, part):
    """
    Load X-ray dataset
    """
    dataset = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for body in os.listdir(folder_path):
                if body == part:
                    body_part = body
                    body_path = os.path.join(folder_path, body)
                    for patient_id in os.listdir(body_path):
                        patient_path = os.path.join(body_path, patient_id)
                        for label_dir in os.listdir(patient_path):
                            label = 'fractured' if label_dir.endswith('positive') else 'normal'
                            label_path = os.path.join(patient_path, label_dir)
                            for img in os.listdir(label_path):
                                img_path = os.path.join(label_path, img)
                                dataset.append({
                                    'body_part': body_part,
                                    'patient_id': patient_id,
                                    'label': label,
                                    'image_path': img_path
                                })
    return dataset

def trainPart(part):
    data = load_path(DATASET_PATH, part)
    labels, filepaths = [], []

    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    images = pd.concat([filepaths, labels], axis=1)

    # Split dataset: 10% test, 90% train (from which 20% for validation)
    train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1)

    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip=True,
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
        validation_split=0.2
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='training'
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='validation'
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )

    pretrained_model = tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )
    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(50, activation='relu')(x)
    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    print("-------Training " + part + "-------")
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]

    history = model.fit(train_images, validation_data=val_images, epochs=25, callbacks=callbacks)

    model.save(os.path.join(OUTPUT_PATH, f"ResNet50_{part}_frac.h5"))
    results = model.evaluate(test_images, verbose=0)
    print(f"{part} Results: {results}")
    print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")

    # Plot accuracy and save
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(os.path.join(OUTPUT_PATH, f"{part}_Accuracy.jpeg"))
    plt.clf()

    # Plot loss and save
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(os.path.join(OUTPUT_PATH, f"{part}_Loss.jpeg"))
    plt.clf()

# Run training for each body part
categories_parts = ["Elbow", "Hand", "Shoulder"]
for category in categories_parts:
    trainPart(category)


Found 3885 validated image filenames belonging to 2 classes.
Found 971 validated image filenames belonging to 2 classes.
Found 540 validated image filenames belonging to 2 classes.
-------Training Elbow-------
Epoch 1/25


C:\Users\ajayk\anaconda3\envs\py310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 371s 6s/step - accuracy: 0.5996 - loss: 0.6796 - val_accuracy: 0.7188 - val_loss: 0.5660
Epoch 2/25
 2/61 ━━━━━━━━━━━━━━━━━━━━ 3:33 4s/step - accuracy: 0.7227 - loss: 0.5446


KeyboardInterrupt

